# Reinforcement Learning: Q-learning en tablas.

Estaremos usando el Gym environment llamado Taxi-V2. Este consiste de un juego muy simple en donde se probaron muchos algoritmos de Reinforcement Learning jerárquico. Se tienen 4 ubicaciones en el ambiente y la meta es recoger al pasajero en una ubicación y llevarlo a otra. El agente puede llevar a cabo 6 acciones: moverse al sur, nort, oeste y este, recoger y dejar al pasajero. Este cuaderno está basado en las siguintes dos fuentes: [Rubik's cube](https://rubikscode.net/2019/06/24/introduction-to-q-learning-with-python-and-open-ai-gym/) y [OpenAI Taxi-V2](https://gym.openai.com/envs/Taxi-v2/).

In [ ]:
#!pip install gym[atari]

In [1]:
import numpy as np
import random
from IPython.display import clear_output
import gym
import time

Veamos como se ven unas iteraciones del juego:

In [2]:
enviroment = gym.make("Taxi-v2").env

enviroment.render()
for i in range(100):
    action = enviroment.action_space.sample()
    clear_output(wait=True)
    enviroment.step(action)
    time.sleep(.5)
    enviroment.render()

print('Número de Estados: {}'.format(enviroment.observation_space.n))
print('Número de Acciones: {}'.format(enviroment.action_space.n))

KeyboardInterrupt: 

In [3]:
# Tamaño de peso para actualizar la tabla de valores Q
alpha = 0.1
# Tasa de descuento temporal
gamma = 0.6
# Probabilidad de exploración
epsilon = 0.1

Genera una matriz (o numpy array de 2D) para guardar los valores "Q"

¿Cuáles son las dos dimensiones que debe de tener?

In [4]:
tabla_q = np.zeros([enviroment.observation_space.n, enviroment.action_space.n])

Define una función que actualice el estimado del valor "q".

¿Qué insumos debe tener?

In [5]:
def actualiza_q(alpha,q_actual,recompensa,gamma,max_V):
    nuevo_q = (1 - alpha) * q_actual + alpha * (recompensa + gamma * max_V)
    return(nuevo_q)

Define la función de exploración vs. aprovechamiento, está debe regresar una acción. 

Ya sea aleatoria para explorar o la mejor posible para "aprovechar" 

In [6]:
def genera_accion(epsilon,tabla_q,estado):
    # Toma acción aprendida o genera una aleatoria para explorar con probabilidad epsilon
    if random.uniform(0, 1) < epsilon:
        accion = enviroment.action_space.sample()
    else:
        accion = np.argmax(tabla_q[estado])
    return(accion)

In [7]:
No_episodios = 100000

for episodio in range(0, No_episodios):
    # Reinicia el ambiente
    estado = enviroment.reset()

    # Inicializa las variables
    recompensa = 0
    termina = False
    
    while not termina:
        # Decide acción a tomar
        accion = genera_accion(epsilon,tabla_q,estado)
        # Implementa la acción    
        sig_estado, recompensa, termina, info = enviroment.step(accion) 
        
        # Actualiza valor q
        q_actual = tabla_q[estado, accion]
        max_V = np.max(tabla_q[sig_estado])
        nuevo_q = actualiza_q(alpha,q_actual,recompensa,gamma,max_V)
        
        # Actualiza tabla de valores q
        tabla_q[estado, accion] = nuevo_q
        estado = sig_estado
    # Imprime progreso
    if (episodio + 1) % 100 == 0:
        clear_output(wait=True)
        print("Episodio: {}".format(episodio + 1))
        enviroment.render()

print("**********************************")
print("Entrenamiento finalizado!\n")
print("**********************************")

Episodio: 100000
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
**********************************
Entrenamiento finalizado!

**********************************


In [8]:
total_epochs = 0
total_penalidades = 0
No_episodios = 100
for _ in range(No_episodios):
    estado = enviroment.reset()
    epochs = 0
    penalidades = 0
    recompensa = 0
    
    termina = False
    
    while not termina:
        accion = np.argmax(tabla_q[estado])
        estado, recompensa, termina, info = enviroment.step(accion)

        if recompensa == -10:
            penalidades += 1

        epochs += 1

    total_penalidades += penalidades
    total_epochs += epochs

print("**********************************")
print("Resultados")
print("**********************************")
print("Epochs por episodio: {}".format(total_epochs / No_episodios))
print("Penalidades por episodio: {}".format(total_penalidades / No_episodios))

**********************************
Resultados
**********************************
Epochs por episodio: 12.14
Penalidades por episodio: 0.0
